In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix,classification_report,ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'sklearn.grid_search'

In [ ]:
data = pd.read_csv(r'/Users/aruni/VScode_Workspace/project/dataset/data.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop(['Unnamed: 32','id'],axis=1,inplace=True)

In [ ]:
y = data['diagnosis']
X = data.drop(['diagnosis'],axis=1)

In [ ]:
y.unique()

In [ ]:
y.value_counts()

In [ ]:
sns.histplot(y)

In [ ]:
#map the diagnosis to 0 and 1
y = y.map({'M':1,'B':0})
y.value_counts()

In [ ]:
X.head()

In [ ]:
sns.pairplot(data,hue='diagnosis',vars=['radius_mean','texture_mean','smoothness_mean','compactness_mean','concavity_mean'])

In [ ]:
sns.heatmap(X.corr())

In [ ]:
def high_corr(data,threshold):
    highly_corr = []
    for i in range(len(data.corr().columns)):
        for j in range(i):
            if abs(data.corr().iloc[i,j]) > threshold:
                print(f'({data.corr().columns[i]},{data.corr().columns[j]}) : {data.corr().iloc[i,j]}')
                highly_corr.append((data.corr().columns[i],data.corr().columns[j]))
    return highly_corr

high_corr_list = high_corr(X,0.95)

In [ ]:
high_corr_list

In [ ]:
X.drop(['perimeter_mean','area_mean','perimeter_se','area_se','perimeter_worst','area_worst'],axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
selector = SelectKBest(f_classif,k=X.shape[1])
selector.fit(X,y)
print(selector.scores_)
for i,score in enumerate(selector.scores_):
    plt.bar(i,score)
plt.xticks(range(X.shape[1]),X.columns,rotation=90)
plt.show()

In [ ]:
X.drop(['fractal_dimension_mean','texture_se','smoothness_se','symmetry_se','fractal_dimension_se',],axis=1,inplace=True)

In [ ]:
X.shape

In [ ]:
X.describe()

In [ ]:
X = StandardScaler().fit_transform(X)

In [ ]:
X_,X_test,Y_,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train,X_val,y_train,y_val = train_test_split(X_,Y_,test_size=0.3,random_state=42)
print(X_train.shape,X_val.shape,X_test.shape)

In [ ]:
model = RandomForestClassifier()
params = {'max_depth':[2,4,10,30,100], 'min_samples_split':[2,5,10,20,50], 'n_estimators':[10,50,100,200]}
best = grid_search(model,params,X_train,y_train)
best

In [ ]:
model = RandomForestClassifier(max_depth=best['max_depth'],min_samples_split=best['min_samples_split'],n_estimators=best['n_estimators']).fit(X_train,y_train)
y_pred = model.predict(X_val)
predection = model.predict(X_test)
print(f'Val Accuracy: {accuracy_score(y_val, y_pred)}')
print(f'Test Accuracy: {accuracy_score(y_test, predection)}')

In [ ]:
print(classification_report(y_test,predection))
cm = confusion_matrix(y_test,predection)
ConfusionMatrixDisplay(cm,display_labels=['B','M']).plot()